Import libraries


In [29]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2
import tensorflow as tf
from tensorflow import keras
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from tensorflow.keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dropout, Flatten, Dense, InputLayer
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, Callback, ReduceLROnPlateau
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers

Uploading files to colab


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving train_v2.csv to train_v2.csv


In [ ]:
import io
train_df = pd.read_csv(io.BytesIO(uploaded['train_v2.csv']))

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving sample_submission_v2.csv to sample_submission_v2.csv


In [ ]:
import io
test_df = pd.read_csv(io.BytesIO(uploaded['sample_submission_v2.csv']))


Hyperparameter Tuning


In [ ]:
DROPOUT_RATE = 0.5
epoch = 22
batch_size = 128
input_shape = (128, 128, 3)
input_size = 128
seed = 42
learning_rate=0.0001


In [ ]:
train_df['image_name'] = train_df['image_name'].astype(str) + '.jpg'
test_df['image_name'] = test_df['image_name'].astype(str) + '.jpg'

test_df['tags'] = test_df['tags'].apply(lambda x: x.split(' '))

print(train_df.head())
print(test_df.head())

    image_name                                       tags
0  train_0.jpg                               haze primary
1  train_1.jpg            agriculture clear primary water
2  train_2.jpg                              clear primary
3  train_3.jpg                              clear primary
4  train_4.jpg  agriculture clear habitation primary road
   image_name                                        tags
0  test_0.jpg  [primary, clear, agriculture, road, water]
1  test_1.jpg  [primary, clear, agriculture, road, water]
2  test_2.jpg  [primary, clear, agriculture, road, water]
3  test_3.jpg  [primary, clear, agriculture, road, water]
4  test_4.jpg  [primary, clear, agriculture, road, water]


Creating a list of all labels in the dataset



In [27]:
labels = []

for tag in train_df['tags'].values:
    label_tag = tag.split(' ')
    for label in label_tag:
      if label not in labels:
        labels.append(label)
        
labels.sort()
print(labels)

['agriculture', 'artisinal_mine', 'bare_ground', 'blooming', 'blow_down', 'clear', 'cloudy', 'conventional_mine', 'cultivation', 'habitation', 'haze', 'partly_cloudy', 'primary', 'road', 'selective_logging', 'slash_burn', 'water']


Model Building


In [35]:
model = Sequential()

# model.add(BatchNormalization())
model.add(Conv2D(32, (3, 3), padding='same', input_shape=(128, 128, 3)))
model.add(Conv2D(32, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(256, (3, 3), padding='same'))
model.add(Conv2D(256, (3, 3), padding='same'))
model.add(Conv2D(256, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(512, (3, 3), padding='same'))
model.add(Conv2D(512, (3, 3), padding='same'))
model.add(Conv2D(512, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(4096))
model.add(Dense(4096))
model.add(Activation('relu'))

model.add(Dense(449, activation='softmax'))
opt = optimizers.RMSprop(learning_rate=0.0001)
model.compile(optimizer=opt,
              loss='categorical_crossentropy',
              metrics=['accuracy'])



I could not go futher becase i was having issues uploading the jpg dataset.

In [36]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_30 (Conv2D)          (None, 128, 128, 32)      896       
                                                                 
 conv2d_31 (Conv2D)          (None, 128, 128, 32)      9248      
                                                                 
 activation_12 (Activation)  (None, 128, 128, 32)      0         
                                                                 
 max_pooling2d_11 (MaxPoolin  (None, 64, 64, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_32 (Conv2D)          (None, 64, 64, 64)        18496     
                                                                 
 conv2d_33 (Conv2D)          (None, 64, 64, 64)        36928     
                                                      

In [ ]:
plt.figure(figsize=(16, 14))
res = [32, 64, 128, 256]
NIMGS = 5

for i in range(len(res)):
    for j in range(NIMGS):
        img = cv2.imread(os.path.join(TRAIN_PATH,df_train['image_name'][j+1]))
        img = cv2.resize(img, (res[i], res[i]))
        plt.subplot(len(res), NIMGS, i*NIMGS+j+1)
        plt.imshow(img)
        plt.title(df_train['tags'][j+1] + "\n" + str(res[i]) + "x" + str(res[i]), rotation=18)
        plt.axis('off')
    
        plt.show()